In [1]:
# Load env variables and create client
import base64, os
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()
Anthropic_API_Key = os.getenv("Anthropic_API_Key")
if not Anthropic_API_Key:
    raise ValueError("Anthropic_API_Key is not set in the environment variables.")
client = Anthropic(api_key=Anthropic_API_Key)

model = "claude-3-7-sonnet-latest"

In [2]:
# Helper functions
from anthropic.types import Message


def add_user_message(messages, message):
    user_message = {
        "role": "user",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(user_message)


def add_assistant_message(messages, message):
    assistant_message = {
        "role": "assistant",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(assistant_message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    thinking=False,
    thinking_budget=1024,
):
    params = {
        "model": model,
        "max_tokens": 4000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if thinking:
        params["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget,
        }

    if tools:
        params["tools"] = tools

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message


def text_from_message(message):
    return "\n".join(
        [block.text for block in message.content if block.type == "text"]
    )

In [3]:
with open("earth.pdf", "rb") as f:
    file_bytes = base64.standard_b64encode(f.read()).decode("utf-8")

messages = []

add_user_message(
    messages,
    [
        {
            "type": "document",
            "source": {
                "type": "base64",
                "media_type": "application/pdf",
                "data": file_bytes,
            },
        },
        {"type": "text", "text": "Summarize the document in one sentence"},
    ],
)

chat(messages)

Message(id='msg_018sXtAhn2Sgaw8JSqJzBidN', content=[TextBlock(citations=None, text='The document is a Wikipedia article about Earth, detailing its physical characteristics, atmosphere, orbit, formation history, and etymology as the third planet from the Sun and the only known astronomical object to harbor life.', type='text')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=9625, output_tokens=44, server_tool_use=None, service_tier='standard'))

## Key Changes from Image Processing

When adapting your image processing code for PDFs, you need to update several elements:

* Change the file extension from .png to .pdf
* Update the variable name from image_bytes to file_bytes for clarity
* Set the type to "document" instead of "image"
* Change the media type to "application/pdf" instead of "image/png"
